### 生成一个小的示例数据集

In [1]:
import numpy as np

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

### 定义功能函数

In [4]:
# creatC1()函数用来构建第一个候选项集的列表C1，即大小为1的所有候选项集的集合列表
def creatC1(dataSet):
    """
    dataSet - 数据集  
    """
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            # 之所以使用列表，是因为Python不能创建只有一个整数的集合
            if not [item] in C1:
                C1.append([item])
    # 排序
    C1.sort()
    # 将列表C1中的每个单元素列表映射到frozenset()，并返回frozenset的列表
    return list(map(frozenset, C1))

In [5]:
# 用于从C1生成L1，另外该函数会返回一个包含支持度值的字典以备后用
def scanD(D,Ck,minSupport):
    """
    D:数据集
    Ck:候选项集列表
    minSupport:最小支持度
    """
    ssCnt = {}
    # 遍历数据集中的所有记录以及C1中的所有候选集。
    # 如果C1中的集合是记录的一部分，则增加字典中对应的计数值
    # 其中，字典中的键为集合
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] =1
                else:
                    ssCnt[can] += 1
    numItems = len(D)
    retList = []
    supportData = {}
    # 循环遍历字典中的每个元素并且计算支持度
    for key in ssCnt:
        support = ssCnt[key] / numItems
        # 如果支持度满足要求，这将字典元素添加到列表中
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList , supportData

### 构建完整的Apriori算法

In [6]:
# aprioriGen()函数用于生成候选项集Ck。
# 例如，该函数以[{0},{1},{2}])作为输入，会生成[{0, 1}, {0, 2}, {1, 2}] 。
# 以[{0,1},{0,2},{0,3},{1,2},{1,3}]作为输入，会生成[{0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}] 。
def aprioriGen(Lk,k):
    """
    Lk:频繁项集列表
    k:项集元素个数
    """
    retList = []
    lenLk = len(Lk)
    # 通过两个for循环，比较Lk中的元素与其他元素
    for i in range(lenLk):
        for j in range(i+1 , lenLk):
            # 取列表中的两个集合进行比较，如果两个集合的前面 k-2 个元素相等，那么就将这两个集合合成一个大小为k的集合
            L1 = list(Lk[i])[:k-2] 
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [7]:
# Apriori算法实现
def apriori(dataSet,minSupport=0.5):
    C1 = creatC1(dataSet)
    # 将数据集转换为集合列表
    D = list(map(set, dataSet))
    # 从C1生成L1
    L1, supportData = scanD(D, C1, minSupport)
    
    L = [L1]
    k=2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2] , k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

### 从频繁项集中挖掘关联规则

In [8]:
# 生成包含可信度的规则列表
def generateRules(L, supportData, minConf=0.7): 
    """
    L - 频繁项集列表
    supportData - 包含那些频繁项集支持数据的字典
    minConf - 最小可信度阈值
    输出：
    bigRuleList - 包含可信度的规则列表
    """
    bigRuleList = []
    # 因为无法从单元素项集中构建关联规则，因此需要从包含两个或更多元素的项集开始规则构建，因此索引从1开始
    for i in range(1,len(L)):
        for freqSet in L[i]:
            # 将项集中的元素转换为集合列表格式，例如，frozenset({2, 3})→[frozenset({2}), frozenset({3})]
            H1 = [frozenset([item]) for item in freqSet]
            # 如果项集中元素多于两个，使用rulesFromConseq()函数从最初的项集中生成更多的关联规则
            if (i>1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            # 如果项集中只有两个元素，使用calcConf()函数计算可信度值
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
                
    return bigRuleList

In [9]:
# 返回一个满足最小可信度要求的规则列表
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    """
    freqSet - 频繁项集
    H - 出现在规则右部的元素列表
    supportData - 包含那些频繁项集支持数据的字典
    brl - 包含可信度的规则列表
    minConf - 最小可信度阈值
    输出：
    prunedH - 复合置信度的关联规则列表
    """
    prunedH = []
    for conseq in H:
        # 计算可信度值，例如，可信度（{3, 5}→{2}） = 支持度（frozenset({2, 3, 5})）/支持度（frozenset({3, 5})）
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf: 
            # 如果某条规则满足最小可信度要求，则将其打印输出
            print(freqSet-conseq,'-->',conseq,'可信度值:',conf)
            brl.append((freqSet-conseq , conseq , conf))
            prunedH.append(conseq)
    return prunedH

In [10]:
# 从最初的项集中生成更多的关联规则
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    """
    freqSet - 频繁项集
    H - 出现在规则右部的元素列表
    supportData - 包含那些频繁项集支持数据的字典
    brl - 包含可信度的规则列表
    minConf - 最小可信度阈值
    """
    # 计算频繁项集的大小
    m = len(H[0])
    if (len(freqSet) > (m + 1)): 
        # 生成H中元素的无重复组合，该结果存储在Hmp1中，这也是下一次迭代的H列表,即创建Hm+1条新候选项集
        Hmp1 = aprioriGen(H, m+1)
        # Hmp1包含所有可能的规则，使用calcConf()函数来测试他们的可信度，以确定规则是否满足要求
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        # 如果不止一条规则满足要求，那么使用Hmp1迭代调用rulesFromConseq()函数来判断是否可以进一步组合这些规则
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

### 算法应用

In [11]:
L, suppData = apriori(dataSet, minSupport=0.5)

In [12]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [13]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [14]:
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) 可信度值: 1.0
frozenset({2}) --> frozenset({5}) 可信度值: 1.0
frozenset({1}) --> frozenset({3}) 可信度值: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [15]:
rules = generateRules(L, suppData, minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) 可信度值: 0.6666666666666666
frozenset({2}) --> frozenset({3}) 可信度值: 0.6666666666666666
frozenset({5}) --> frozenset({3}) 可信度值: 0.6666666666666666
frozenset({3}) --> frozenset({5}) 可信度值: 0.6666666666666666
frozenset({5}) --> frozenset({2}) 可信度值: 1.0
frozenset({2}) --> frozenset({5}) 可信度值: 1.0
frozenset({3}) --> frozenset({1}) 可信度值: 0.6666666666666666
frozenset({1}) --> frozenset({3}) 可信度值: 1.0
frozenset({5}) --> frozenset({2, 3}) 可信度值: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) 可信度值: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) 可信度值: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]